<a href="https://colab.research.google.com/github/pruhnav/LoadForecasting/blob/main/Price_forecast_XGBoost%2C_BiLSTM%2C_LSTM%2C_GRU%2C_Transformer%2C_CNN%2C_Hybrid_48h_12t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy Price Prediction with XGBoost & BiLSTM

# Task for reza
- Make the progam to forecast 24 hours of prices at once.
- Change the input from t = 24, 24*3, 24*5, 24*7, 24*9, 24*11.
- Input for others 24*7, Output t = 24.
- See link: https://ieeexplore-ieee-org.ezaccess.libraries.psu.edu/stamp/stamp.jsp?tp=&arnumber=10214317
- Create Fig. 10 for MAE (Mean Absulute Error) and RMSE (Root Mean Squared Error) for Test data set (Proposed Hybrid Model).
- Two cases comparison (Actual vs Predcited): Best and Worst week.
- Whole year (2023) Actual vs Predicted and (Residual = Actual - Predicted)


In [ ]:
from google.colab import files

# Open file upload dialog
#uploaded = files.upload()

In [ ]:
import pandas as pd

# Load the uploaded Excel file
df = pd.read_excel('day_ahead_energy_prices.xlsx')

# Display the first few rows of the DataFrame
print(df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'day_ahead_energy_prices.xlsx'

In [ ]:
df.head()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


In [ ]:
# Assuming `df` is your DataFrame containing 'Date' and 'Price'
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [ ]:


# Data Preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Price'].values.reshape(-1, 1))

# Create data structure for BiLSTM
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

def prepare_data(data, look_back=48, look_ahead=1):
  X, y = [], []
  for i in range(len(data) - look_back - look_ahead + 1):
    X.append(data[i:(i + look_back)])
    y.append(data[(i + look_back):(i + look_back + look_ahead)])
  return np.array(X), np.array(y)


#X, y = create_dataset(scaled_data)
X, y = prepare_data(scaled_data)
X.shape,y.shape

In [ ]:
X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape for BiLSTM input

# Split into training and testing sets based on year
train_df = df[df.index.year <= 2022]
test_df = df[df.index.year == 2023]
train_scaled = scaler.transform(train_df['Price'].values.reshape(-1, 1))
test_scaled = scaler.transform(test_df['Price'].values.reshape(-1, 1))

X_train, y_train = prepare_data(train_scaled)
X_test, y_test = prepare_data(test_scaled)


In [ ]:

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
EPOCH = 3 # change here

In [ ]:

# Build BiLSTM Model
def build_bilstm_model(input_shape, lstm_units=64, dropout_rate=0.2):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(lstm_units, return_sequences=False))(inputs)

    x = Dense(128, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(12)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

bilstm_model = build_bilstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

In [ ]:
# prompt: create xgboost model function

def create_xgboost_model(X_train, y_train):
  """Creates and trains an XGBoost model.

  Args:
    X_train: Training input data.
    y_train: Training target data.

  Returns:
    Trained XGBoost model.
  """
  xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
  xgb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train.reshape(y_train.shape[0], -1))
  return xgb_model


In [ ]:
# Train BiLSTM

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
bilstm_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCH,
    batch_size=32,
    callbacks=[early_stopping]
)

In [ ]:
# prompt: see layers of bilstm_model

bilstm_model.summary()


In [ ]:
# prompt: apply the bilstm model and give the mse, rmse, mae, smape, and r^2 score

# Predict using BiLSTM
y_pred_bilstm = bilstm_model.predict(X_test)

# Inverse transform the predictions and actual values
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_bilstm_inv = scaler.inverse_transform(y_pred_bilstm.reshape(-1, 1))

# Calculate evaluation metrics
mse_bilstm = mean_squared_error(y_test_inv, y_pred_bilstm_inv)
rmse_bilstm = np.sqrt(mse_bilstm)
mae_bilstm = mean_absolute_error(y_test_inv, y_pred_bilstm_inv)

# SMAPE calculation
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

smape_bilstm = smape(y_test_inv, y_pred_bilstm_inv)

r2_bilstm = r2_score(y_test_inv, y_pred_bilstm_inv)

print(f"BiLSTM - MSE: {mse_bilstm:.4f}")
print(f"BiLSTM - RMSE: {rmse_bilstm:.4f}")
print(f"BiLSTM - MAE: {mae_bilstm:.4f}")
print(f"BiLSTM - SMAPE: {smape_bilstm:.4f}")
print(f"BiLSTM - R^2 Score: {r2_bilstm:.4f}")


In [ ]:
# prompt: apply the xgboost model and give the mse, rmse, mae, smape, and r^2 score

# Train XGBoost
xgb_model = create_xgboost_model(X_train, y_train)

# Predict using XGBoost
y_pred_xgb = xgb_model.predict(X_test.reshape(X_test.shape[0], -1))


# Inverse transform the predictions and actual values
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_xgb_inv = scaler.inverse_transform(y_pred_xgb.reshape(-1, 1))

# Calculate evaluation metrics
mse_xgb = mean_squared_error(y_test_inv, y_pred_xgb_inv)
rmse_xgb = np.sqrt(mse_xgb)
mae_xgb = mean_absolute_error(y_test_inv, y_pred_xgb_inv)
smape_xgb = smape(y_test_inv, y_pred_xgb_inv)
r2_xgb = r2_score(y_test_inv, y_pred_xgb_inv)

print(f"XGBoost - MSE: {mse_xgb:.4f}")
print(f"XGBoost - RMSE: {rmse_xgb:.4f}")
print(f"XGBoost - MAE: {mae_xgb:.4f}")
print(f"XGBoost - SMAPE: {smape_xgb:.4f}")
print(f"XGBoost - R^2 Score: {r2_xgb:.4f}")


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

lstm_model = Sequential()
lstm_model.add(LSTM(64, activation='relu', input_shape=(48, 1)))  # assuming univariate input
lstm_model.add(Dense(12))  # 6-hour forecast
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary()

# Train the model
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)


In [ ]:
from tensorflow.keras.layers import GRU

gru_model = Sequential()
gru_model.add(GRU(64, activation='relu', input_shape=(48, 12)))
gru_model.add(Dense(1))
gru_model.compile(optimizer='adam', loss='mse')
gru_model.summary()

gru_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)


In [ ]:
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model

inputs = Input(shape=(48, 12))
x = Dense(64)(inputs)
x = MultiHeadAttention(num_heads=2, key_dim=32)(x, x)
x = LayerNormalization()(x)
x = Dropout(0.1)(x)
x = GlobalAveragePooling1D()(x)
outputs = Dense(1)(x)

transformer_model = Model(inputs, outputs)
transformer_model.compile(optimizer='adam', loss='mse')
transformer_model.summary()

transformer_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)


In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

cnn_model = Sequential()
cnn_model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(48, 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(1))
cnn_model.compile(optimizer='adam', loss='mse')
cnn_model.summary()

cnn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 100 * np.mean(diff)

def evaluate_model(model, X_test, y_test, scaler=None, model_name="Model"):
    y_pred = model.predict(X_test)

    y_test = np.squeeze(y_test)
    y_pred = np.squeeze(y_pred)

    # Inverse scale if scaler is given
    if scaler:
        y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(y_test.shape)
        y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).reshape(y_pred.shape)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    smape_score = smape(y_test, y_pred)

    print(f"{model_name} - MSE: {mse:.4f}")
    print(f"{model_name} - RMSE: {rmse:.4f}")
    print(f"{model_name} - MAE: {mae:.4f}")
    print(f"{model_name} - SMAPE: {smape_score:.2f}%")
    print(f"{model_name} - R^2 Score: {r2:.4f}")


In [ ]:
evaluate_model(lstm_model, X_test, y_test, scaler=scaler, model_name="LSTM")
evaluate_model(gru_model, X_test, y_test, scaler=scaler, model_name="GRU")
evaluate_model(transformer_model, X_test, y_test, scaler=scaler, model_name="Transformer")
evaluate_model(cnn_model, X_test, y_test, scaler=scaler, model_name="CNN")


### Task - HYBRID MODEL
input 24, 48, 72, 96

change the feature to 30, 60, 90, 120

In [ ]:
# Extract Features from BiLSTM
intermediate_layer_model = Model(inputs=bilstm_model.input, outputs=bilstm_model.layers[-2].output)
bilstm_features_train = intermediate_layer_model.predict(X_train)
bilstm_features_test = intermediate_layer_model.predict(X_test)

In [ ]:
bilstm_features_train.shape,bilstm_features_test.shape

In [ ]:
# sMAPE Calculation
def calculate_smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0  # handle the case where both prediction and actual are zero
    return 100 * np.mean(diff)



# Modified MAPE Calculation (exclude zero values in actual prices)
def calculate_mape(y_true, y_pred):
    non_zero_mask = y_true != 0  # Exclude zero actual values
    y_true = y_true[non_zero_mask]
    y_pred = y_pred[non_zero_mask]
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

In [ ]:
evaluation_data = pd.DataFrame(columns=['Model', 'MAPE', 'RMSE', 'MAE', 'R2'])
def evaluation_metrics(y_test_original, y_pred_original, model_name="No Model"):
  mae = mean_absolute_error(y_test_original, y_pred_original)
  rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
  mape = mean_absolute_percentage_error(y_test_original, y_pred_original)
  r2 = r2_score(y_test_original, y_pred_original)


  #print(f"Mean Absolute Percentage Error (MAPE): {mape*100}")
  #print(f"RMSE: {rmse}")
  #print(f"Mean Absolute Error (MAE): {mae}")
  #print(f"R-squared (R2): {r2}")
  # Create a new row as a dictionary
  new_row = {'Model': model_name, 'MAPE': mape*100 , 'RMSE': rmse , 'MAE': mae , 'R2': r2 }
  global evaluation_data;
  # Add the new row using the loc indexer
  evaluation_data.loc[model_name] = new_row
  #print(evaluation_data)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# Train XGBoost with BiLSTM Features
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42)
y_train_reshaped = y_train.reshape(y_train.shape[0], -1)  # Reshape to (num_samples, 24)
xgb_model.fit(bilstm_features_train, y_train_reshaped, verbose=True)



# Save Actual vs Predicted Prices in Excel
#results_df = pd.DataFrame({
 #   'Date': test_df.index[-len(xgb_predictions):],
  #  'Actual Price': test_actual.flatten(),
   # 'Predicted Price': xgb_predictions_rescaled.flatten()
#})
#results_df.to_excel('actual_vs_predicted.xlsx', index=False)
#print("Actual vs Predicted saved to 'actual_vs_predicted.xlsx'.")




In [ ]:

# Make Predictions with XGBoost
xgb_predictions = xgb_model.predict(bilstm_features_test)
xgb_predictions_rescaled = scaler.inverse_transform(xgb_predictions.reshape(-1, 6))

# Rescale Test Data
test_actual = scaler.inverse_transform(y_test.reshape(-1, 6))

In [ ]:
evaluation_metrics(test_actual, xgb_predictions_rescaled, f"XGBoost_look_back_48_hour")
evaluation_data

In [ ]:
test_actual

In [ ]:
xgb_predictions_rescaled

In [ ]:
# prompt: find the difference between test_actual and xgb_predictions_rescaled where  every item comparison difference is more than 10

# Find differences greater than 10
differences = np.abs(test_actual - xgb_predictions_rescaled)
indices = np.where(differences > 10)

# Print the indices and values where the difference is greater than 10
print("Indices where the difference is greater than 10:")
print(indices)

print("\nDifferences greater than 10:")
print(differences[indices])

# Access the corresponding values from test_actual and xgb_predictions_rescaled
print("\nValues from test_actual:")
print(test_actual[indices])

print("\nValues from xgb_predictions_rescaled:")
xgb_predictions_rescaled[indices]


In [ ]:
# Evaluation Metrics
mse = mean_squared_error(test_actual, xgb_predictions_rescaled)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_actual, xgb_predictions_rescaled)
r2 = r2_score(test_actual, xgb_predictions_rescaled)

smape = calculate_smape(test_actual, xgb_predictions_rescaled)

mape = mean_absolute_percentage_error(test_actual, xgb_predictions_rescaled)

print(f"Hybrid Model Evaluation:\nMAE: {mae}, RMSE: {rmse}, sMAPE: {smape}%, MAPE: {mape}, R^2: {r2}, MSE: {mse}")


In [ ]:
# prompt: draw a line graph based test_actual and xgb_predictions_rescaled data every 24 data

import matplotlib.pyplot as plt

# Assuming xgb_predictions_rescaled and test_actual are already defined

# Plotting every 24th data point
plt.figure(figsize=(12, 6))
plt.plot(test_actual.flatten()[10000:20000:24], label='Actual')
plt.plot(xgb_predictions_rescaled.flatten()[10000:20000:24], label='Predicted')
plt.xlabel('Time Step (every 24)')
plt.ylabel('Energy Price')
plt.title('Actual vs Predicted Energy Prices (every 24 data points)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Visualization
SHOW_TEST_VALUES = 400

plt.figure(figsize=(15, 5))
plt.plot(test_df.index[-SHOW_TEST_VALUES:], test_actual[-SHOW_TEST_VALUES:], label='Actual Price')
plt.plot(test_df.index[-SHOW_TEST_VALUES:], xgb_predictions_rescaled[-SHOW_TEST_VALUES:], label='Predicted Price')
plt.title('Energy Price Prediction using Hybrid XGBoost-BiLSTM')
plt.xlabel('Date')
plt.ylabel('Price')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))#'%Y-%m-%d %H:%M
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=3))
plt.gcf().autofmt_xdate()
plt.legend()
plt.savefig("Energy Price Prediction(proposed).pdf", bbox_inches='tight')
plt.show()

In [ ]:
print("Extracted Features (Training):")
print(bilstm_features_train)

In [ ]:
training_features_df = pd.DataFrame(bilstm_features_train)
# training_features_df.to_excel('bilstm_features_train.xlsx', index=False) %%


To apply SHAP (SHapley Additive exPlanations) to a hybrid model combining BiLSTM and XGBoost, you need to follow a structured approach depending on how you integrate both models. Typically, this hybrid approach consists of two stages:

Feature Extraction using BiLSTM: A BiLSTM (Bidirectional Long Short-Term Memory) network processes sequential data and extracts meaningful features.
Prediction using XGBoost: The extracted features are then passed to an XGBoost model for final predictions.
## Applying SHAP to this Hybrid Model
Since SHAP explanations depend on the model type, you need to determine whether to explain the BiLSTM model, the XGBoost model, or both

In [ ]:
import shap
# Explain XGBoost predictions using SHAP values
explainer = shap.Explainer(xgb_model)
shap_values = explainer(bilstm_features_test)

# Visualize SHAP values
shap.summary_plot(shap_values, bilstm_features_test, plot_type="bar",max_display=10,show=False)
plt.savefig("shap_summary_plot_bilstm_features_bar_graph.pdf", bbox_inches='tight')
plt.show()

In [ ]:

shap.summary_plot(shap_values, bilstm_features_test,max_display=10,show=False)
plt.savefig("shap_summary_plot_bilstm_features.pdf", bbox_inches='tight')
plt.show()

In [ ]:
X_train_temp = X_train.reshape(X_train.shape[0],-1)
X_test_temp = X_test.reshape(X_test.shape[0],-1)
num_features = X_train_temp.shape[1]

# Generate column names (Modify this based on actual feature names)
feature_names = [f"t-{60-i}" for i in range(num_features)]  # Default names
X_train_df = pd.DataFrame(X_train_temp, columns=feature_names)
X_test_df = pd.DataFrame(X_test_temp, columns=feature_names)

In [ ]:
NUM_ROWS = 500
explainer_bilstm = shap.Explainer(bilstm_model, X_train_df[:NUM_ROWS])  # Sample subset for efficiency
shap_values_bilstm = explainer_bilstm(X_test_df[:NUM_ROWS])




In [ ]:
#shap.summary_plot(shap_values_bilstm, X_test_df, plot_type="bar",max_display=10,show=False)

#plt.savefig("shap_summary_plot_bilstm_average_impact.pdf", bbox_inches='tight')
#plt.show()


In [ ]:
# Visualize the SHAP values for BiLSTM

shap.summary_plot(shap_values_bilstm, X_test_df[:NUM_ROWS], max_display=10,show=False)


plt.savefig("shap_summary_plot_bilstm_impact_on_model_output.pdf", bbox_inches='tight')
plt.show()

#t-1, t-2 .....

In [ ]:
shap.plots.waterfall(shap_values_bilstm[1],show=False)


plt.savefig("shap_summary_plot_bilstm_waterfall.pdf", bbox_inches='tight')
plt.show()


In [ ]:
shap.plots.bar(shap_values_bilstm,show=False)
plt.savefig("shap_summary_plot_bilstm_mean_shap_value.pdf", bbox_inches='tight')
plt.show()

In [ ]:

explainer_bilstm = shap.Explainer(bilstm_model, X_train_df[:10])
shap_values_bilstm = explainer_bilstm(X_test_df[:10])

# Instead of using explainer_bilstm.model.predict, use your original model:
expected_value = bilstm_model.predict(X_train_df[:10]).mean()

shap.decision_plot(expected_value, shap_values_bilstm[1].values, X_test_df.columns,show=False)
plt.savefig("shap_summary_plot_bilstm_model_output.pdf", bbox_inches='tight')
plt.show()

In [ ]:
shap.plots.beeswarm(shap_values_bilstm)